columns of MFP from start, MFP from high density, MFP from popped LLS, ceil LLS, id of associated halo, center of halo, mass of halo, rvir of halo, association


In [1]:
from scipy import integrate
import matplotlib.pyplot as plt
import os
import numpy as np
from matplotlib import pyplot as plt      
import pandas as pd
from scipy import special                 
import array
import scipy as sp
import scipy.interpolate
import re
import pickle as pickle
from concurrent.futures import ProcessPoolExecutor

In [2]:
def unPackRawFile(raw_path):
    """
    - unpacks the .raw file. Not used for the neural network.
    """
    y = np.loadtxt(raw_path, skiprows=6)
    distance = y[:,0]
    pec_vel = y[:,1]
    temp = y[:,2]
    HI_density = y[:,3]

   
    return distance, pec_vel, temp, HI_density

def getPos(path_LOS,linenumber=8):
    """
    the start position of the LOS is given inside each file, (in the comments)
    this function parses the comments to get that information
    """
    f = open(path_LOS)
    x = f.readlines()[linenumber]
    answer = re.search('\(([^)]+)', x).group(1)
    arr = np.array(answer.split(','),dtype=float)
    return arr

def getDir(path_LOS,linenumber=8):
    """
    the direction of the LOS is given inside each file, (in the comments)
    this function parses the comments to get that information
    """
    f = open(path_LOS)
    x = f.readlines()[linenumber]
    answer = re.search('\(([^)]+)', x.split(', ')[1]).group(1)
    arr = np.array(answer.split(','),dtype=float)
    return arr


def convertSphereToCart(theta, phi):
    "converts a unit vector in spherical to cartesian, needed for getGalaxies"
    return np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])
def change_to_redshift(distance,redshift):
    '''change Cmpc to redshift, start at the redshift of the box'''
    distance_redshift = []
    distance_redshift.append(redshift)
    distance_difference = redshift
    for i in range(len(distance)-1):
        distance_difference -= (distance[i+1] - distance[i]) / 100 * .23
        distance_redshift.append(distance_difference)
    return distance_redshift

def change_to_wavelength(redshift_distance,redshift):
    start_wavelength = 912
    wavelength = []
    for i in range(len(redshift_distance)):
        wavelength.append(912*(1+redshift_distance[i])/(1+redshift))
    return wavelength

In [3]:
def distance_measurement(redshift, result):
    '''modified code from  a_distance_measurement to measure individual mfp for each of the los'''
    total_distance = []
    distance_average = 0
    distance_median = 0
    factor = 1/(redshift+1)/.68
    for i in result:
        for k in range(1,len(i[0])):
            if(np.power(np.e,np.multiply(-1,i[0][k])) >0.3679):
                distance_average += 0.0667
        for k in range(1,len(i[1])):
            if(np.power(np.e,np.multiply(-1,i[1][k])) >0.3679):
                distance_median += 0.0667
        total_distance.append([np.multiply(distance_average,factor),np.multiply(distance_median,factor)])
        distance_average = 0
        distance_median = 0
    return total_distance

In [8]:
raw_path = '/mnt/c/Users/jiawe/research/mfp/box_data/LOS/40_box/MFP_0.1356/'+'los.00' + '{0:03}'.format(0) +'.raw'
distance, pec_vel, temp, HI_density = unPackRawFile(raw_path)

Measure individual mfp, then take the median and the 75 and 25

In [ ]:
def individual_mfp(redshift,file_path):
    '''this calculates the individual mfp of the los'''
    sigma_value = 6.3e-18
    mfp = []
    for i in range(1000):
        raw_path = file_path + 'los.00' + '{0:03}'.format(i) +'.raw'
        distance, pec_vel, temp, HI_density, gas_density, gas_metallicity = unPackRawFile(raw_path)
                    #changing the distance from chimp to cemeters
        distance_redshift = change_to_redshift(distance,redshift)
        factor = 1/(redshift+1)*3.086e24 /0.68
        distance_cm = distance *factor
        k = 1
        wavelength = change_to_wavelength(distance_redshift,redshift)
        while k< len(distance) -2:
            total_optical_depth = [0]
            mfp_temp = 0
            distance_graph = []
            HI_graph = []
            optical_depth_stuff = []
            sigma = []
            while(k< len(distance) -2):
                distance_graph.append(distance_cm[k])
                HI_graph.append(HI_density[k])
                sigma.append(sigma_value)
                total_optical_depth = integrate.cumtrapz(np.multiply(HI_graph,sigma),distance_graph,initial =0)
                optical_depth_stuff.append(total_optical_depth[-1])
                mfp_temp += distance[k] - distance[k-1]
                k+=1
            x = []
            for p in distance[:-3]:
                x.append(p)
            mfp.append([x,optical_depth_stuff])

array([6.740000e-03, 1.587000e-02, 2.856000e-02, ..., 9.996987e+01,
       9.998491e+01, 9.999760e+01])